# Correlation analysis, CCA and feature importance: Unmet needs

## Outcome variables:
- Maternal mortality rate: rate_maternal_mortality
- Under 5 mortality rate: rate_under5y_mortality
- Antenatal coverage (ANC): prop_antenatal_coverage
- Proportion of unmet contraceptive need: prop_unmet_need_family_planing
- ORS: 

In [1]:
import re
import collections
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.cross_decomposition import CCA
from sklearn.preprocessing import StandardScaler

%matplotlib inline

In [2]:
def remove_miss_vars(input_df):
    df = input_df.copy(deep=True)
    remove_list = []
    for var in df.columns:
        if any(df[var].isna()):
            remove_list.append(var)
    return df.drop(remove_list, axis=1)

def impute_miss_vars(input_df):
    df = input_df.copy(deep=True)
    for var in df.columns:
        if any(df[var].isna()):
            df[var].fillna(df[var].mean, inplace=True)
    return df

def intersect_dfs(input_df1, input_df2):
    df1 = input_df1.copy(deep=True)
    df2 = input_df2.copy(deep=True)
    subset_var = list(set(list(df1.columns)).intersection(set(list(df2.columns))))
    return df1[subset_var], df2[subset_var]

## STEP 1: Import data and data processing: remove absolute and remove missing values

In [3]:
DATA2011 = '/Users/edinhamzic/Symphony/wb_bangladesh/Bangladesh/output/all/all2011.csv'
DATA2016 = '/Users/edinhamzic/Symphony/wb_bangladesh/Bangladesh/output/all/all2016.csv'
DHIS2_VARS = '/Users/edinhamzic/Symphony/wb_bangladesh/Bangladesh/output/all/DHIS_Rate_Absolute.csv'
OUT = '/Users/edinhamzic/Symphony/wb_bangladesh/Bangladesh/output/all/'

In [4]:
d2011 = pd.read_csv(DATA2011)
d2016 = pd.read_csv(DATA2016)
dhis2vars = pd.read_csv(DHIS2_VARS)
print(dhis2vars.shape)
tmp = dhis2vars[dhis2vars['Rate_Absolute'] == 'Absolute']
print(tmp.shape)
vars_remove = list(tmp['Full_name'])

(349, 4)
(279, 4)


In [5]:
d2011 = d2011.drop(vars_remove, axis=1)
d2016 = d2016.drop(vars_remove, axis=1)

In [6]:
d2011.shape
d2011 = d2011.set_index(['DistrictName'])
print(d2011.shape)
d2011 = d2011.drop(['DistrictGeo'], axis=1)
print(d2011.shape)
subset_vars = [var for var, var_type in zip(d2011.dtypes.index, d2011.dtypes) if str(var_type) != 'object'] 
d2011 = d2011[subset_vars]
d2011 = d2011.fillna(d2011.mean())
#d2011 = remove_miss_vars(input_df=d2011)
print(d2011.shape)
d2011.head()

(64, 188)
(64, 187)
(64, 187)


,TT1_Mother0-11MChildren,Measles_Children23M,TT4_Mother0-11MChildren,OPV3_Children12M,OPV2_Children23M,OPV3_Children23M,PENTA2_Children12M,TT3_Mother0-11MChildren,PENTA3_Children23M,VitACoverage_Children12-59M,...,prop_pop_rural.1,prop_institutional_delivery,prop_current_contraceptive,prop_female_head,prop_antenatal_care4.,prop_unmet_need_family_planing,prop_pop_women.1,dependency_ratio.1,prop_registered_under5,sex_ratio.1
DistrictName,,,,,,,,,,,,,,,,,,,,,
Bagerhat,90.0,88.1,52.9,93.3,96.0,93.3,96.0,71.4,85.6,96.7,...,11.48,30.66,65.52,11.29,26.56,7.89,44.85,74.37,32.84,93.31
Bandarban,89.0,82.3,54.3,88.8,90.8,89.4,90.2,72.4,84.4,83.3,...,0.00,0.00,45.45,11.29,0.00,27.27,127.03,107.50,31.25,84.44
Barguna,99.0,88.1,47.6,94.7,98.6,95.4,98.6,72.4,87.9,96.7,...,6.48,10.56,72.36,2.97,20.97,11.14,25.33,70.73,41.40,105.44
Barisal,97.1,86.8,52.4,94.5,98.1,94.5,98.1,77.1,86.9,79.0,...,8.22,21.53,64.33,5.10,26.53,13.31,24.48,75.53,38.53,89.02
Bhola,98.6,85.0,67.6,93.0,96.5,93.0,95.8,85.7,86.3,87.1,...,7.40,9.64,68.40,5.65,29.51,10.81,26.74,80.16,21.19,94.76


In [7]:
d2016.shape
d2016 = d2016.set_index(['DistrictName'])
print(d2016.shape)
d2016 = d2016.drop(['DistrictGeo'], axis=1)
print(d2016.shape)
subset_vars = [var for var, var_type in zip(d2016.dtypes.index, d2016.dtypes) if str(var_type) != 'object'] 
d2016 = d2016[subset_vars]
d2016 = d2016.fillna(d2016.mean())
# d2016 = remove_miss_vars(input_df=d2016)
print(d2016.shape)
d2016.head()

(64, 188)
(64, 187)
(64, 187)


,TT1_Mother0-11MChildren,Measles_Children23M,TT4_Mother0-11MChildren,OPV3_Children12M,OPV2_Children23M,OPV3_Children23M,PENTA2_Children12M,TT3_Mother0-11MChildren,PENTA3_Children23M,VitACoverage_Children12-59M,...,prop_pop_rural.1,prop_institutional_delivery,prop_current_contraceptive,prop_female_head,prop_antenatal_care4.,prop_unmet_need_family_planing,prop_pop_women.1,dependency_ratio.1,prop_registered_under5,sex_ratio.1
DistrictName,,,,,,,,,,,,,,,,,,,,,
Bagerhat,98.1,92.2,61.9,90.8,97.1,91.0,96.8,85.3,91.0,82.0,...,14.38,26.49,67.82,4.17,25.13,11.00,39.86,56.82,37.56,94.78
Bandarban,94.8,89.8,79.6,87.8,94.8,87.8,94.8,86.1,87.8,84.1,...,0.00,75.00,63.64,25.49,75.00,13.64,77.52,45.16,40.00,84.93
Barguna,98.8,94.9,64.4,93.0,98.3,93.6,97.9,88.4,93.6,96.8,...,9.48,27.54,73.30,8.06,47.46,8.22,28.64,68.24,25.10,87.82
Barisal,100.0,97.1,79.3,95.5,99.3,96.0,99.3,96.7,96.0,100.0,...,28.48,46.33,64.26,6.99,32.39,10.08,30.97,59.68,23.40,94.74
Bhola,100.0,96.6,79.0,94.5,99.8,94.5,99.8,94.1,94.5,98.4,...,9.99,11.27,67.28,3.53,15.62,10.01,26.58,69.35,17.64,101.71


## Renaming variables

In [8]:
var_names = pd.read_csv("/Users/edinhamzic/Symphony/wb_bangladesh/Bangladesh/output/all/IndicatorsNames_2011_2016.csv")
var_names.head()

if all(d2016.columns == d2011.columns):
    print(True)
    if all([var1==var2 for var1,var2 in zip(list(d2016.columns), list(var_names['Indicators 2011']))]):
        new_names = []
        for key, var in enumerate(d2016.columns):
            new_names.append(var_names.loc[key,'Description'])
        d2011.columns = new_names
        d2016.columns = new_names

True


## STEP 2: Outcome variables: Unmet needs

### Proportion of unmet contraceptive need

In [9]:
print(f"Mean: {round(d2011['DHS: Unmet need for family planning'].mean(),2)}",
      f" Standard deviation: {round(d2011['DHS: Unmet need for family planning'].std(),2)}")

Mean: 13.56  Standard deviation: 6.31


In [10]:
print(f"Mean: {round(d2016['DHS: Unmet need for family planning'].mean(),2)}",
      f" Standard deviation: {round(d2016['DHS: Unmet need for family planning'].std(),2)}")

Mean: 11.94  Standard deviation: 5.53


### Scaled and normalized data


In [11]:
print(d2011.shape)
drop_columns = []
for var in d2011.columns:
    if "index" in var:
        drop_columns.append(var)
drop_columns = list(set(drop_columns))
print(drop_columns)
d2011.drop(drop_columns, inplace=True, axis=1)
print(d2011.shape)

(64, 187)
[]
(64, 187)


In [12]:
print(d2016.shape)
drop_columns = []
for var in d2016.columns:
    if "index" in var:
        drop_columns.append(var)
drop_columns = list(set(drop_columns))
print(drop_columns)
d2016.drop(drop_columns, inplace=True, axis=1)
print(d2016.shape)

(64, 187)
[]
(64, 187)


In [13]:
s_data2011 = StandardScaler().fit_transform(d2011)
s_data2011 = pd.DataFrame(s_data2011, columns=d2011.columns)
print(s_data2011.shape)
s_data2011 = remove_miss_vars(input_df=s_data2011)
print(s_data2011.shape)

(64, 187)
(64, 187)


/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/anaconda3/lib/python3.7/site-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


In [14]:
d2016[d2016==np.inf]=np.nan
d2016.fillna(d2016.mean(), inplace=True)
s_data2016 = StandardScaler().fit_transform(d2016)
s_data2016 = pd.DataFrame(s_data2016, columns=d2016.columns)
print(s_data2016.shape)
s_data2016 = remove_miss_vars(input_df=s_data2016)
print(s_data2016.shape)


(64, 187)
(64, 187)


/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/anaconda3/lib/python3.7/site-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


### Performing PCA on both years 2011 and 2016: Preliminary

In [15]:
pca = PCA(.95)
pca2011 = pca.fit(s_data2011)
print(pca.n_components_)

40


In [16]:
pca = PCA(.95)
pca2016 = pca.fit(s_data2016)
print(pca.n_components_)

42


## Performing Canonical Correlation Analysis (CCA) and correlation analysis for 2011
- The aim of this part is to identify variables highly correlated with proportion of unmet contraceptive need

### Canonical Correlation Analysis (CCA) 2011

In [17]:
cca_unmetneeds = CCA(copy=True, max_iter=1000, n_components=40, scale=True, tol=1e-06)
cca_unmetneeds.fit(s_data2011.drop('DHS: Unmet need for family planning', axis=1),
                 s_data2011['DHS: Unmet need for family planning'])
print(cca_unmetneeds.score(s_data2011.drop('DHS: Unmet need for family planning', axis=1),
                         s_data2011['DHS: Unmet need for family planning']))

0.9240645421535875


/anaconda3/lib/python3.7/site-packages/sklearn/cross_decomposition/pls_.py:291: UserWarning: Y residual constant at iteration 1
  warnings.warn('Y residual constant at iteration %s' % k)


In [18]:
CCA_coeff_unmetneeds = pd.DataFrame({'Indicators': list(s_data2011.drop('DHS: Unmet need for family planning', axis=1).columns),
                                   'CCA_coeff': cca_unmetneeds.coef_[:,0],
                                   'CCA_coeff_abs': np.absolute(cca_unmetneeds.coef_[:,0]),})
CCA_coeff_unmetneeds.sort_values(by='CCA_coeff_abs', ascending=False).head()
CCA_coeff_unmetneeds = CCA_coeff_unmetneeds[CCA_coeff_unmetneeds['CCA_coeff_abs'] > 0.1]
display(CCA_coeff_unmetneeds.head())
print(f"Number of indicators from the original file with coefficent values above 0.1 is {CCA_coeff_unmetneeds.shape[0]}")
CCA_coeff_unmetneeds.to_csv(OUT+'/cca_unmet_needs_2011.csv', index=False, index_label=False)

,Indicators,CCA_coeff,CCA_coeff_abs
32,Death rate,-0.161607,0.161607
36,Proportion of women population,0.122904,0.122904
37,Dependency ratio,0.176366,0.176366
38,Sex ratio,-0.118884,0.118884
39,Proportion of live births,0.105973,0.105973


Number of indicators from the original file with coefficent values above 0.1 is 25


### Correlation analysis 2011

In [19]:
import scipy.stats  as stats
all(s_data2011.columns == s_data2016.columns)

True

In [20]:
corr2011 = s_data2011.corr()

In [21]:
unmet_needs_2011 = corr2011[['DHS: Unmet need for family planning']]
unmet_needs_2011['abs_prop_unmet_need_family_planing'] = np.absolute(unmet_needs_2011['DHS: Unmet need for family planning'])
unmet_needs_2011 = unmet_needs_2011.sort_values(by='abs_prop_unmet_need_family_planing', ascending=False)
unmet_needs_2011.drop('DHS: Unmet need for family planning',axis=0, inplace=True)
unmet_needs_2011.reset_index(inplace=True)
corr_pvalues = []
for var in unmet_needs_2011['index']:
    pvalue = stats.pearsonr(s_data2011[var], s_data2011['DHS: Unmet need for family planning'])[1]
    corr_pvalues.append(pvalue)
unmet_needs_2011['p_value'] = corr_pvalues
unmet_needs_2011 = unmet_needs_2011[unmet_needs_2011['p_value'] < 0.1]
unmet_needs_2011.sort_values(by = 'abs_prop_unmet_need_family_planing', ascending=False)
display(unmet_needs_2011.head())
print(f"Number of indicators from the original file with correlation coefficient values above 0.1 is {unmet_needs_2011.shape[0]}")
unmet_needs_2011.to_csv(OUT+'/corr_unmet_needs_2011.csv', index=False, index_label=False)

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/anaconda3/lib/python3.7/site-packages/scipy/stats/stats.py:3010: RuntimeWarning: invalid value encountered in double_scalars
  r = r_num / r_den


,index,DHS: Unmet need for family planning,abs_prop_unmet_need_family_planing,p_value
0,DHS: Proportion of use of current contraceptiv...,-0.831776,0.831776,1.708276e-17
1,Dependency ratio,0.633349,0.633349,1.949960e-08
2,"Distributed IUDs out of total (thana), %",0.617231,0.617231,5.572988e-08
3,"Distributed IUDs out of total (month, thana), %",0.617231,0.617231,5.572988e-08
4,DHS: Sex ratio (males / females),-0.548085,0.548085,2.760155e-06


Number of indicators from the original file with correlation coefficient values above 0.1 is 80


### Combine results

In [22]:
unmet_needs_vars_2011 = list(set(CCA_coeff_unmetneeds['Indicators']).intersection(set(unmet_needs_2011['index'])))
print(unmet_needs_vars_2011)
len(unmet_needs_vars_2011)

['DHS: Proportion of use of current contraceptive methods', 'DHS: Dependency ratio', 'Postpartum hemorrhage among admitted patient in EmONC, %', 'Functional radiant warmer, %', 'Sex ratio', 'Dependency ratio', 'Newborn who received postnatal care within two days of birth at facility, %', 'Proportion of women population', 'Proportion of cases with postpartum hemorrhage among admitted patient in EmONC', 'Death rate']


10

In [23]:
unmet_needs_vars_2011 = list(set(CCA_coeff_unmetneeds['Indicators']).union(set(unmet_needs_2011['index'])))
len(unmet_needs_vars_2011)

95

## Performing Canonical Correlation Analysis (CCA) and correlation analysis for 2016
- The aim of this part is to identify variables highly correlated with proportion of unmet contraceptive need

### Canonical Correlation Analysis (CCA) 2016

In [24]:
cca_unmetneeds = CCA(copy=True, max_iter=500, n_components=40, scale=True, tol=1e-06)
cca_unmetneeds.fit(s_data2016.drop('DHS: Unmet need for family planning', axis=1),
                 s_data2016['DHS: Unmet need for family planning'])
print(cca_unmetneeds.score(s_data2016.drop('DHS: Unmet need for family planning', axis=1),
                         s_data2016['DHS: Unmet need for family planning']))

0.9739703051754869


/anaconda3/lib/python3.7/site-packages/sklearn/cross_decomposition/pls_.py:291: UserWarning: Y residual constant at iteration 1
  warnings.warn('Y residual constant at iteration %s' % k)


In [25]:
CCA_coeff_unmetneeds = pd.DataFrame({'Indicators': list(s_data2016.drop('DHS: Unmet need for family planning', axis=1).columns),
                                   'CCA_coeff': cca_unmetneeds.coef_[:,0],
                                   'CCA_coeff_abs': np.absolute(cca_unmetneeds.coef_[:,0]),})
CCA_coeff_unmetneeds.sort_values(by='CCA_coeff_abs', ascending =False).head()
CCA_coeff_unmetneeds = CCA_coeff_unmetneeds[CCA_coeff_unmetneeds['CCA_coeff_abs'] > 0.1]
display(CCA_coeff_unmetneeds.head())
print(f"Number of indicators from the original file with coefficent values above 0.1 is {CCA_coeff_unmetneeds.shape[0]}")
CCA_coeff_unmetneeds.to_csv(OUT+'/cca_unmet_needs_2016.csv', index=False, index_label=False)

,Indicators,CCA_coeff,CCA_coeff_abs
1,"Crude coverage measles children 23 months, %",0.127010,0.127010
9,"Vitamin A coverage children 12-59 months, %",0.117877,0.117877
25,Proportion of women (15-45 years),-0.155142,0.155142
27,Fertility rate,0.139743,0.139743
34,Proportion of rural deaths,-0.112983,0.112983


Number of indicators from the original file with coefficent values above 0.1 is 26


### Correlation analysis 2016

In [26]:
import scipy.stats  as stats
all(s_data2011.columns == s_data2016.columns)

True

In [27]:
corr2016 = s_data2016.corr()

In [28]:
unmet_needs_2016 = corr2016[['DHS: Unmet need for family planning']]
unmet_needs_2016['abs_prop_unmet_need_family_planing'] = np.absolute(unmet_needs_2016['DHS: Unmet need for family planning'])
unmet_needs_2016 = unmet_needs_2016.sort_values(by='abs_prop_unmet_need_family_planing', ascending=False)
unmet_needs_2016.drop('DHS: Unmet need for family planning',axis=0, inplace=True)
unmet_needs_2016.reset_index(inplace=True)
corr_pvalues = []
for var in unmet_needs_2016['index']:
    pvalue = stats.pearsonr(s_data2016[var], s_data2016['DHS: Unmet need for family planning'])[1]
    corr_pvalues.append(pvalue)
unmet_needs_2016['p_value'] = corr_pvalues
unmet_needs_2016 = unmet_needs_2016[unmet_needs_2016['p_value'] < 0.1]
unmet_needs_2016.sort_values(by = 'abs_prop_unmet_need_family_planing', ascending=False)
display(unmet_needs_2016.head())
print(f"Number of indicators from the original file with correlation coefficient values above 0.1 is {unmet_needs_2016.shape[0]}")
unmet_needs_2016.to_csv(OUT+'/corr_unmet_needs_2016.csv', index=False, index_label=False)

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/anaconda3/lib/python3.7/site-packages/scipy/stats/stats.py:3010: RuntimeWarning: invalid value encountered in double_scalars
  r = r_num / r_den


,index,DHS: Unmet need for family planning,abs_prop_unmet_need_family_planing,p_value
0,DHS: Proportion of use of current contraceptiv...,-0.839687,0.839687,4.346822e-18
1,DHS: Dependency ratio,0.652078,0.652078,5.319055e-09
2,DHS: Proportion of females as head of household,0.426293,0.426293,4.442170e-04
3,Performed permanent methods out of total (than...,0.378648,0.378648,2.033911e-03
4,Performed permanent methods out of total (mont...,0.378611,0.378611,2.036142e-03


Number of indicators from the original file with correlation coefficient values above 0.1 is 27


### Combine results

In [29]:
unmet_needs_vars_2016 = list(set(CCA_coeff_unmetneeds['Indicators']).intersection(set(unmet_needs_2016['index'])))
print(unmet_needs_vars_2016)
len(unmet_needs_vars_2016)

['DHS: Proportion of use of current contraceptive methods', 'DHS: Dependency ratio', 'DHS: Proportion of C-sections', 'DHS: Proportion of institutional deliveries', 'DHS: Proportion of females as head of household', 'TT vial wastage rate']


6

In [30]:
unmet_needs_vars_2016 = list(set(CCA_coeff_unmetneeds['Indicators']).union(set(unmet_needs_2016['index'])))
len(unmet_needs_vars_2016)

47

## Intersect variables

In [31]:
print(len(unmet_needs_vars_2011))
print(len(unmet_needs_vars_2016))
unmet_needs_vars_union = list(set(unmet_needs_vars_2011).union(set(unmet_needs_vars_2016)))
unmet_needs_vars_inter = list(set(unmet_needs_vars_2011).intersection(set(unmet_needs_vars_2016)))
print(len(unmet_needs_vars_union))
print(len(unmet_needs_vars_inter))

95
47
115
27


## Performing HDBSCAN or Kmean clustering: Unmet needs for contraceptive rate 

In [32]:
import os
import re
import glob
import conda
import hdbscan
import operator
import itertools
import numpy as np
import pandas as pd
import seaborn as sns
from config import Config
from collections import Counter
from matplotlib import pyplot as plt
conda_file_dir = conda.__file__
conda_dir = conda_file_dir.split('lib')[0]
proj_lib = os.path.join(os.path.join(conda_dir, 'share'), 'proj')
os.environ["PROJ_LIB"] = proj_lib
from mpl_toolkits.basemap import Basemap
from matplotlib.patches import Polygon
from matplotlib.lines import Line2D
from matplotlib.collections import PatchCollection
from sklearn.cluster import KMeans

In [33]:
def evaluate_hdbscan(input_df, min_samples, min_cluster_size, 
                     output, cluster_selection_method, 
                     fmin_samples, fmin_cluster_size,
                     prune=False, plot=True):
    samples = list(itertools.product(min_samples, min_cluster_size))
    counter = 0
    models = pd.DataFrame(columns=['min_samples',
                                   'min_cluster_size',
                                   'num_clusters_including_unclustered',
                                   'percent_of_unclustered_geos',
                                   'percent_of_maxclass',],index=range(len(samples)))
    #geo = input_df['index']
    #input_df = input_df.drop('index', axis=1)
    df = input_df.copy(deep=True)
    for iteration in samples:
        model = hdbscan.HDBSCAN(min_samples=int(iteration[0]), 
                                min_cluster_size=int(iteration[1]), 
                                metric='euclidean', 
                                algorithm='best',
                                cluster_selection_method=cluster_selection_method, prediction_data=False).fit(df)
        models.loc[counter,'min_cluster_size'] = iteration[1]
        models.loc[counter, 'min_samples'] = iteration[0]
        models.loc[counter, 'num_clusters_including_unclustered'] = len(Counter(model.labels_))
        tmp_dict = dict(Counter(model.labels_))
        total = sum([v for k,v in tmp_dict.items()])
        tmp_dict = {k:round(v/total*100,2) for k,v in tmp_dict.items()}
        try:
            models.loc[counter, 'percent_of_unclustered_geos'] = tmp_dict.pop(-1)
        except KeyError as error:
            models.loc[counter, 'percent_of_unclustered_geos'] = 0 
        if len(tmp_dict) > 1:
            models.loc[counter, 'percent_of_maxclass'] = tmp_dict[max(tmp_dict.items(), key=operator.itemgetter(1))[0]]
        else:
            models.loc[counter, 'percent_of_maxclass'] = 100
        counter += 1
    if prune:
        out_model = hdbscan.HDBSCAN(min_samples=int(fmin_samples), 
                                min_cluster_size=int(fmin_cluster_size), 
                                metric='euclidean', 
                                algorithm='best',
                                cluster_selection_method=cluster_selection_method, prediction_data=False).fit(df)

    else:
        out_model = None

    if plot:
        plt.rcParams['figure.figsize'] = [20,10]
        plt.plot(models['num_clusters_including_unclustered'], label='Number of clusters including unclustered')
        plt.plot(models['percent_of_unclustered_geos'], label='Percent of unclustered geographies')
        plt.plot(models['percent_of_maxclass'], label='Size of larges cluster (%)')
        plt.xlabel("Iterations", fontsize=20)
        plt.ylabel("Value", fontsize=20)
        plt.savefig(os.path.split(output)[1] + "/finetune_parameteres.jpeg")
        plt.legend()
        plt.show()
    del(input_df, df)
    return models, out_model

In [34]:
OUT = '/Users/edinhamzic/Symphony/wb_bangladesh/Bangladesh/output/all/'

## Evaluate clustering method: HDBSCAN - leaf - 2011

In [35]:
tmp, out = evaluate_hdbscan(input_df=s_data2011[unmet_needs_vars_union], 
                       min_samples=Config.tune_min_sample, 
                       min_cluster_size=Config.tune_min_cluster,
                       output=OUT, cluster_selection_method ='leaf',
                       fmin_samples=3, fmin_cluster_size=12,
                       prune=True, plot=False)

In [36]:
tmp = tmp[tmp['num_clusters_including_unclustered'] <10]
tmp.sort_values('percent_of_unclustered_geos', ascending=True).head(10)

,min_samples,min_cluster_size,num_clusters_including_unclustered,percent_of_unclustered_geos,percent_of_maxclass
1,1,3,4,54.69,35.94
49,2,3,4,54.69,35.94
0,1,2,8,59.38,18.75
48,2,2,8,59.38,18.75
192,5,2,3,82.81,14.06
293,7,7,1,100,100
292,7,6,1,100,100
291,7,5,1,100,100
290,7,4,1,100,100
289,7,3,1,100,100


## Evaluate clustering method: HDBSCAN - eom - 2011

In [37]:
tmp, out = evaluate_hdbscan(input_df=s_data2011[unmet_needs_vars_union], 
                       min_samples=Config.tune_min_sample, 
                       min_cluster_size=Config.tune_min_cluster,
                       output=OUT, cluster_selection_method ='eom',
                       fmin_samples=3, fmin_cluster_size=12,
                       prune=True, plot=False)

In [38]:
tmp = tmp[tmp['num_clusters_including_unclustered'] <10]
tmp.sort_values('percent_of_unclustered_geos', ascending=True).head(10)

,min_samples,min_cluster_size,num_clusters_including_unclustered,percent_of_unclustered_geos,percent_of_maxclass
0,1,2,4,14.06,78.12
48,2,2,4,14.06,78.12
1,1,3,3,17.19,78.12
49,2,3,3,17.19,78.12
192,5,2,3,82.81,14.06
293,7,7,1,100,100
292,7,6,1,100,100
291,7,5,1,100,100
290,7,4,1,100,100
289,7,3,1,100,100


## Evaluate clustering method: HDBSCAN - leaf - 2016

In [39]:
tmp, out = evaluate_hdbscan(input_df=s_data2016[unmet_needs_vars_inter], 
                       min_samples=Config.tune_min_sample, 
                       min_cluster_size=Config.tune_min_cluster,
                       output=OUT, cluster_selection_method ='leaf',
                       fmin_samples=3, fmin_cluster_size=12,
                       prune=True, plot=False)

In [40]:
tmp = tmp[tmp['num_clusters_including_unclustered'] <10]
tmp.sort_values('percent_of_unclustered_geos', ascending=True).head(10)

,min_samples,min_cluster_size,num_clusters_including_unclustered,percent_of_unclustered_geos,percent_of_maxclass
2,1,4,3,70.31,23.44
1,1,3,6,73.44,6.25
96,3,2,4,82.81,9.38
97,3,3,3,85.94,9.38
48,2,2,4,87.5,6.25
144,4,2,3,87.5,7.81
145,4,3,3,87.5,7.81
241,6,3,3,89.06,6.25
240,6,2,3,89.06,6.25
192,5,2,3,90.62,6.25


## Evaluate clustering method: HDBSCAN - eom - 2016

In [41]:
tmp, out = evaluate_hdbscan(input_df=s_data2016[unmet_needs_vars_union], 
                       min_samples=Config.tune_min_sample, 
                       min_cluster_size=Config.tune_min_cluster,
                       output=OUT, cluster_selection_method ='eom',
                       fmin_samples=3, fmin_cluster_size=12,
                       prune=True, plot=False)

In [42]:
tmp = tmp[tmp['num_clusters_including_unclustered'] <10]
tmp.sort_values('percent_of_unclustered_geos', ascending=True).head(10)

,min_samples,min_cluster_size,num_clusters_including_unclustered,percent_of_unclustered_geos,percent_of_maxclass
0,1,2,3,26.56,70.31
48,2,2,3,26.56,70.31
1,1,3,3,51.56,43.75
49,2,3,3,51.56,43.75
144,4,2,3,73.44,23.44
2,1,4,3,78.12,14.06
3,1,5,3,78.12,14.06
51,2,5,3,78.12,14.06
50,2,4,3,78.12,14.06
96,3,2,3,87.5,6.25


# Clustering with K-means

### Clustering Union List

In [43]:
kmeans_model = KMeans(n_clusters=4, random_state=0).fit(s_data2011[unmet_needs_vars_union])
predicted2016 = kmeans_model.predict(s_data2016[unmet_needs_vars_union])
d2011['cluster'] = kmeans_model.labels_
d2016['cluster'] = predicted2016

In [44]:
unmet_needs_vars_union.append('cluster')
d2011[unmet_needs_vars_union].to_csv(OUT+'/clusters_unmet_needs_vars_union_2011.csv')
d2016[unmet_needs_vars_union].to_csv(OUT+'/clusters_unmet_needs_vars_union_2016.csv')
print(d2011[unmet_needs_vars_union].shape)
print(d2016[unmet_needs_vars_union].shape)

(64, 116)
(64, 116)


In [45]:
print(Counter(d2011['cluster']))

Counter({1: 28, 3: 20, 0: 10, 2: 6})


In [46]:
print(Counter(d2016['cluster']))

Counter({1: 29, 3: 22, 0: 7, 2: 6})


### Clustering Intersection List

In [47]:
d2011 = d2011.drop('cluster', axis=1)
d2016 = d2016.drop('cluster', axis=1)

In [48]:
kmeans_model = KMeans(n_clusters=4, random_state=0).fit(s_data2011[unmet_needs_vars_inter])
predicted2016 = kmeans_model.predict(s_data2016[unmet_needs_vars_inter])
d2011['cluster'] = kmeans_model.labels_
d2016['cluster'] = predicted2016

In [49]:
print(Counter(d2011['cluster']))

Counter({2: 22, 1: 18, 0: 12, 3: 12})


In [50]:
print(Counter(d2016['cluster']))

Counter({2: 27, 1: 16, 3: 11, 0: 10})


In [51]:
print(d2011[unmet_needs_vars_inter].shape)
print(d2016[unmet_needs_vars_inter].shape)

(64, 27)
(64, 27)


In [52]:
unmet_needs_vars_inter.append('cluster')
d2011[unmet_needs_vars_inter].to_csv(OUT+'/clusters_unmet_needs_vars_intersect_2011.csv')
d2016[unmet_needs_vars_inter].to_csv(OUT+'/clusters_unmet_needs_vars_intersect_2016.csv')
print(d2011[unmet_needs_vars_inter].shape)
print(d2016[unmet_needs_vars_inter].shape)

(64, 28)
(64, 28)


In [53]:
tmp1 = d2011[unmet_needs_vars_inter]
tmp1 = tmp1.reset_index(col_level='DistricName')
tmp1['year'] = 2011
tmp2 = d2016[unmet_needs_vars_inter]
tmp2 = tmp2.reset_index(col_level='DistricName')
tmp2['year'] = 2016
tmp = pd.concat([tmp1, tmp2], axis=0)
print(tmp.shape)
tmp.to_csv(OUT+'clusters_unmet_needs_vars_intersect_all.csv', index=False)

(128, 30)


In [54]:
tmp1 = d2011[unmet_needs_vars_union]
tmp1 = tmp1.reset_index(col_level='DistricName')
tmp1['year'] = 2011
tmp2 = d2016[unmet_needs_vars_union]
tmp2 = tmp2.reset_index(col_level='DistricName')
tmp2['year'] = 2016
tmp = pd.concat([tmp1, tmp2], axis=0)
print(tmp.shape)
tmp.to_csv(OUT+'clusters_unmet_needs_vars_union_all.csv', index=False)

(128, 118)
